<center><font size=3 style="color:#BA4A00"><strong>关联规则挖掘</strong></font></center>

## 1. 导入所需的库和数据集

In [1]:
# 安装机器学习扩展库mlxtend
! pip install mlxtend

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
You should consider upgrading via the '/usr/local/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
# 使用pandas读取并查看数据集
df = pd.read_csv('./data/GroceryStoreDataSet.csv', names = ['products'], sep = ',')
df.head()

products
0             MILK,BREAD,BISCUIT
1  BREAD,MILK,BISCUIT,CORNFLAKES
2            BREAD,TEA,BOURNVITA
3           JAM,MAGGI,BREAD,MILK
4              MAGGI,TEA,BISCUIT

In [4]:
df.shape

(20, 1)

## 2. 对数据集进行格式转换

In [5]:
# 将商品交易记录转化为嵌套列表
data = list(df["products"].apply(lambda x:x.split(",") ))

In [6]:
data

[['MILK', 'BREAD', 'BISCUIT'],
 ['BREAD', 'MILK', 'BISCUIT', 'CORNFLAKES'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['JAM', 'MAGGI', 'BREAD', 'MILK'],
 ['MAGGI', 'TEA', 'BISCUIT'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['MAGGI', 'TEA', 'CORNFLAKES'],
 ['MAGGI', 'BREAD', 'TEA', 'BISCUIT'],
 ['JAM', 'MAGGI', 'BREAD', 'TEA'],
 ['BREAD', 'MILK'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'COCK'],
 ['BREAD', 'SUGER', 'BISCUIT'],
 ['COFFEE', 'SUGER', 'CORNFLAKES'],
 ['BREAD', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['TEA', 'MILK', 'COFFEE', 'CORNFLAKES']]

使用`TransactionEncoder`，将列表转换为One-Hot编码的布尔列表，每条交易记录中，True表示购买某商品，False表示未购买某商品

In [7]:
from mlxtend.preprocessing import TransactionEncoder
a = TransactionEncoder()
a_data = a.fit(data).transform(data)

In [8]:
# a_data是值为True或False的布尔数组
# a_data

In [9]:
df = pd.DataFrame(a_data,columns=a.columns_)
df

BISCUIT  BOURNVITA  BREAD   COCK  COFFEE  CORNFLAKES    JAM  MAGGI   MILK  \
0      True      False   True  False   False       False  False  False   True   
1      True      False   True  False   False        True  False  False   True   
2     False       True   True  False   False       False  False  False  False   
3     False      False   True  False   False       False   True   True   True   
4      True      False  False  False   False       False  False   True  False   
5     False       True   True  False   False       False  False  False  False   
6     False      False  False  False   False        True  False   True  False   
7      True      False   True  False   False       False  False   True  False   
8     False      False   True  False   False       False   True   True  False   
9     False      False   True  False   False       False  False  False   True   
10     True      False  False   True    True        True  False  False  False   
11     True      False  False   True    True        True  False  False  False   
12    False       True  False  False    True       False  False  False  False   
13    False      False   True   True    True       False  False  False  False   
14     True      False   True  False   False       False  False  False  False   
15    False      False  False  False    True        True  False  False  False   
16    False       True   True  False   False       False  False  False  False   
17    False      False   True  False    True       False  False  False  False   
18    False      False   True  False    True       False  False  False  False   
19    False      False  False  False    True        True  False  False   True   

    SUGER    TEA  
0   False  False  
1   False  False  
2   False   True  
3   False  False  
4   False   True  
5   False   True  
6   False   True  
7   False   True  
8   False   True  
9   False  False  
10  False  False  
11  False  False  
12   True  False  
13  False  False  
14   True  False  
15   True  False  
16   True  False  
17   True  False  
18   True  False  
19  False   True

## 3. 应用Apriori算法

`mlxtend`包中Apriori模型的所有参数均可调节，可使用`help`函数查看`apriori`模型的用法和参数。

In [10]:
# help(apriori)

In [11]:
# 将最小支持度设为0.2
freq_df = apriori(df, min_support = 0.2, use_colnames = True, verbose = 1)
freq_df

Processing 42 combinations | Sampling itemset size 3


support              itemsets
0      0.35             (BISCUIT)
1      0.20           (BOURNVITA)
2      0.65               (BREAD)
3      0.40              (COFFEE)
4      0.30          (CORNFLAKES)
5      0.25               (MAGGI)
6      0.25                (MILK)
7      0.30               (SUGER)
8      0.35                 (TEA)
9      0.20      (BISCUIT, BREAD)
10     0.20         (BREAD, MILK)
11     0.20        (BREAD, SUGER)
12     0.20          (BREAD, TEA)
13     0.20  (CORNFLAKES, COFFEE)
14     0.20       (COFFEE, SUGER)
15     0.20          (MAGGI, TEA)

置信度值设置为0.6，即，在购买商品X的条件下，有0.6的概率会购买商品Y

In [12]:
df_rules = association_rules(freq_df, metric = "confidence", min_threshold = 0.6)
df_rules

antecedents consequents  antecedent support  consequent support  support  \
0        (MILK)     (BREAD)                0.25                0.65      0.2   
1       (SUGER)     (BREAD)                0.30                0.65      0.2   
2  (CORNFLAKES)    (COFFEE)                0.30                0.40      0.2   
3       (SUGER)    (COFFEE)                0.30                0.40      0.2   
4       (MAGGI)       (TEA)                0.25                0.35      0.2   

   confidence      lift  leverage  conviction  
0    0.800000  1.230769    0.0375        1.75  
1    0.666667  1.025641    0.0050        1.05  
2    0.666667  1.666667    0.0800        1.80  
3    0.666667  1.666667    0.0800        1.80  
4    0.800000  2.285714    0.1125        3.25

## 4. 结果分析支持商业决策

以上述结果中的第四（索引值为3）条规则为例分析如下：

- 方糖在所有销售记录中的比率占30%；

- 咖啡在所有销售记录中的比率占40%；

- 方糖+咖啡组合在所有销售记录中的比率占20%；

- 在购买方糖的顾客中，有约67%的顾客也会购买咖啡；

- 购买糖的用户可能会比不购买糖的用户多购买8%的咖啡；

- 方糖和咖啡的关联度可用conviction值衡量（1.8）。

如果商品集合X和Y一起购买的频率高，关联性强，可采取以下几个步骤增加利润：

- 通过产品组合改善交叉销售，如顾客购买了电脑后，可提醒其购买鼠标；

- 更改商店布局，将高度关联商品放在一起以提高销售额；

- 为销量较低的产品设计促销活动，以提高销售额；

- 对关联产品，提供组合购买折扣，如，购买电脑后，加够某款鼠标只需9.9元。